# Problem 6

Consider the Adult Income dataset. The ML model is aimed to predict whether the income will exceed 50k or not. 

You need to first preprocess the dataset. Next, you need to build a machine learning model to predict individual's income.

Implement all the questions and upload your code named `{yourlastname}_hw2.ipynb` to Canvas.

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_squared_error
import math


In [3]:
# load dataset
data = pd.read_csv('adult.csv')
data

,age,hours_per_week,workclass,education,marital_status,occupation,race,gender,income
0,39,40,Government,Bachelors,Single,White-Collar,White,Male,0
1,50,13,Self-Employed,Bachelors,Married,White-Collar,White,Male,0
2,38,40,Private,HS-grad,Divorced,Blue-Collar,White,Male,0
3,53,40,Private,School,Married,Blue-Collar,Other,Male,0
4,28,40,Private,Bachelors,Married,Professional,Other,Female,0
...,...,...,...,...,...,...,...,...,...
32556,27,38,Private,Assoc,Married,Service,White,Female,0
32557,40,40,Private,HS-grad,Married,Blue-Collar,White,Male,1
32558,58,40,Private,HS-grad,Widowed,White-Collar,White,Female,0
32559,22,20,Private,HS-grad,Single,White-Collar,White,Male,0


## (5 points) Transform the categorical features into the `one-hot-encoding` format

In [4]:
# Need to transform workclass, education, marital_status, occupation, race, gender, and income. 
# Use the get_dummies() function. 
data_dummies = pd.get_dummies(data)
data_dummies

,age,hours_per_week,income,workclass_Government,workclass_Other/Unknown,workclass_Private,workclass_Self-Employed,education_Assoc,education_Bachelors,education_Doctorate,...,occupation_Blue-Collar,occupation_Other/Unknown,occupation_Professional,occupation_Sales,occupation_Service,occupation_White-Collar,race_Other,race_White,gender_Female,gender_Male
0,39,40,0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,1
1,50,13,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,1,0,1
2,38,40,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
3,53,40,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
4,28,40,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,38,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,1,0
32557,40,40,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
32558,58,40,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,1,0
32559,22,20,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


## (5 points) Normalize the continuous features 

In [8]:
# Need to normalize hours_per_week and age
temp_df = data[["age", "hours_per_week"]] # Isolate the two features 
x = temp_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
temp_df_scaled = min_max_scaler.fit_transform(x)
temp_df_scaled_final = pd.DataFrame(temp_df_scaled, columns=["age", "hours_per_week"])
#temp_df_scaled_final

# Append back to the enconded data frame
temp_data = data_dummies.drop(["age", "hours_per_week"], axis = 1)
data_final = pd.concat([temp_data, temp_df_scaled_final], axis = 1)
#data_final

## (5 points) Split the dataset into `train_X`, `train_y`, `test_X`, `test_y`

Use the first 20,000 data points as the training set, and the rest as the testing set. DO NOT RANDOMLY SLIT THE TRAIN/TEST DATA.
- If you use `sklearn.model_selection.train_test_split`, you should set the parameter `shuffle=False`.

In [20]:
# Get the first 20,000 rows. 
data_first = data_final.iloc[0:20000]
#data_first

# Split into train_X, train_y, test_X, test_y.
# Fist, isolate the "y" (income) and "x" features.
y = data_first[["income"]]
x = data_first.drop(["income"], axis = 1)

# Split
train_X, train_y, test_X, test_y = model_selection.train_test_split(x, y, test_size=0.2, shuffle=False)
train_X

,workclass_Government,workclass_Other/Unknown,workclass_Private,workclass_Self-Employed,education_Assoc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Prof-school,...,occupation_Professional,occupation_Sales,occupation_Service,occupation_White-Collar,race_Other,race_White,gender_Female,gender_Male,age,hours_per_week
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0.301370,0.397959
1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0.452055,0.122449
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0.287671,0.397959
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0.493151,0.397959
4,0,0,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,1,0,0.150685,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0.630137,0.397959
15996,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0.561644,0.500000
15997,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,1,0,0.698630,0.142857
15998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0.328767,0.397959


## (10 points) Build a ML model to predict income in the testing set

You should report the accuracy score in the *testing set*. To obtain full credit, you need to achieve accuracy score higher than 82.5% on the testing set. Please briefly describe your approach.

*(5 bonus point if your accuracy score is higher than 83.5%)*

In [7]:
'''
I want to use feature selection using recursive feature elimination.
This part runs the feature selection 29 times for each feature and lets me choose 
which features to remove. 
'''

error_lst = []
for i in range(1,30):
    x = data_first.drop(["income"], axis = 1)
    y = data_first[["income"]]
    rfe_selector = RFE(estimator=LogisticRegression(),n_features_to_select = i, step = 1)
    rfe_selector.fit(x, y)
    # Make new df with just the important features
    cols = list(x.columns[rfe_selector.get_support()])
    x = x[cols]
    # Split again
    train_X, train_y, test_X, test_y = model_selection.train_test_split(x, y, test_size=0.2, shuffle=False)
    train_X
    # Make Linear Regression Model 
    model = LogisticRegression(fit_intercept=True)
    model.fit(train_X, test_X)
    pred = model.predict(train_y)
    # Make a new df with the actual and predicted variables 
    pred_y = pd.DataFrame(pred, columns = ["Predicted"])
    test_y = test_y.reset_index(drop = True)
    compare = pd.concat([pred_y, test_y], axis = 1)
    error_lst.append([i,accuracy_score(compare["Predicted"], compare["income"])])


/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

KeyboardInterrupt: 

In [21]:
# additional feature engineering if applicable
'''
11 features gave the best score.
'''
rfe_selector = RFE(estimator=LogisticRegression(),n_features_to_select = 11, step = 1)
rfe_selector.fit(x, y)
# Make new df with just the important features
cols = list(x.columns[rfe_selector.get_support()])
x = x[cols]
x = x.drop(["occupation_Service"], axis = 1)
# Split again
train_X, train_y, test_X, test_y = model_selection.train_test_split(x, y, test_size=0.2, shuffle=False)

In [22]:
# build your ML model here

# Make Linear Regression Model 
model = LogisticRegression(fit_intercept=True)
model.fit(train_X, test_X)
pred = model.predict(train_y)

/Users/josiahkim/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [23]:
# evaluate your model on the testing set here

# Make a new df with the actual and predicted variables 
pred_y = pd.DataFrame(pred, columns = ["Predicted"])
test_y = test_y.reset_index(drop = True)
compare = pd.concat([pred_y, test_y], axis = 1)
# SCORE
print("The accuracy score is:", accuracy_score(compare["Predicted"], compare["income"]))

The accuracy score is: 0.82575
